<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/capstone%20/Working%20RNN/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd

# Geospatial processing packages
import geopandas as gpd

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Loading dataframe on current active/and closed coal mines

In [143]:
#join in overall info
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')
mines = pd.DataFrame(mines)
mines = mines[['company',
               'coalmine_op_status',
               'mine_name',
               'permit_id',
               'state_tribe_mine_id',
               'national_id',
               'coal_bed_names',
               'inspectable_unit_status',
                'post_smcra',
               'reported_area',
               'permit_application_type',
               'permit_application_date',
               'permit_approval_date',
               'edit_date',
               'area_mine',
               'contour',
               'mountaintop',
               'steep_slope',
               'highwall',
               'auger',
               'contact']].drop_duplicates()

#replace empty strings with NAN
mines = mines.replace(r'^\s*$', np.nan, regex=True)

### Loading dataframes from google earth engine queries

In [133]:
directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['year'] = directory_files[z]
    df_file['year'] = df_file['year'].str.slice(3, 7)
    df_file['end_date'] = df_file['year']+'-07-31'
    df_file['start_date'] = df_file['year']+'-07-01'
    df_file['start_date'] = pd.to_datetime(df_file['start_date'])
    df_file['end_date'] = pd.to_datetime(df_file['end_date'])

    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df = pd.concat([df, df_file])

    z+=1


### aggregating to averages of land class(the probability) classes

In [134]:
#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file
#419bdf	water, #397d49	trees, #88b053	grass, #7a87c6	flooded_vegetation, #e49635	crops, #dfc35a	shrub_and_scrub, #c4281b	built, #a59b8f	bare, #b39fe1	snow_and_ice

label0 = df[['orig_perm_id', 'Company','mine_id','water', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date','Company']).mean().reset_index()
label1 = df[['orig_perm_id', 'Company','mine_id', 'trees', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label2 = df[['orig_perm_id', 'Company','mine_id', 'grass', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label3 = df[['orig_perm_id', 'Company','mine_id','flooded_vegetation', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date', 'Company']).mean().reset_index()
label4 = df[['orig_perm_id', 'Company','mine_id', 'crops', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index().drop_duplicates()
label5 = df[['orig_perm_id', 'Company','mine_id', 'shrub_and_scrub', 'start_date']].groupby(['orig_perm_id', 'start_date', 'mine_id','Company']).mean().reset_index()
label6 = df[['orig_perm_id', 'Company','mine_id', 'built', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label7 = df[['orig_perm_id', 'Company','mine_id', 'bare', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label8 = df[['orig_perm_id', 'Company','mine_id', 'snow_and_ice', 'start_date']].groupby(['orig_perm_id','start_date','mine_id', 'Company']).mean().reset_index()

label0 =label0.drop_duplicates()
label1 =label1.drop_duplicates()
label2 =label2.drop_duplicates()
label3 =label3.drop_duplicates()
label4 =label4.drop_duplicates()
label5 =label5.drop_duplicates()
label6 =label6.drop_duplicates()
label7 =label7.drop_duplicates()
label8 =label8.drop_duplicates()

group_df = df[['orig_perm_id', 'start_date', 'mine_id', 'Company', 'State']].groupby(['orig_perm_id', 'start_date', 'mine_id', 'State']).count().reset_index()
group_df = group_df.drop_duplicates()
group_df = group_df.rename(columns={"Company": "Observations"})
df = pd.merge(df, group_df, on=['orig_perm_id', 'start_date', 'mine_id', 'State'], how='left').drop_duplicates()
df

,ï»¿,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,...,snow_and_ice,label,Company,mine_id,orig_perm_id,year,end_date,start_date,State,Observations
0,3,20230718T161839_20230718T162101_T17SLC,-82.962111,38.040957,1689697971141,0.032271,0.742175,0.033115,0.028178,0.029994,...,0.035817,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
1,9,20230718T161839_20230718T162101_T17SLC,-82.963908,38.042754,1689697971141,0.032551,0.741282,0.033707,0.028543,0.030174,...,0.035345,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
2,15,20230718T161839_20230718T162101_T17SLC,-82.962111,38.042754,1689697971141,0.031584,0.732819,0.034490,0.028255,0.029593,...,0.033416,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
3,19,20230705T160829_20230705T162114_T17SLC,-82.960315,38.042754,1688574175051,0.024305,0.746942,0.039173,0.024682,0.038829,...,0.035887,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
4,21,20230718T161839_20230718T162101_T17SLC,-82.960315,38.042754,1689697971141,0.030797,0.746335,0.035748,0.026772,0.032971,...,0.033623,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358802,12,20170726T160859_20170726T161854_T17SMB,-81.955998,37.766073,1501085934130,0.080749,0.049291,0.047412,0.040685,0.050573,...,0.045043,6.0,BELVA COAL COMPANY,00000000000000000869,S009482,2017,2017-07-31,2017-07-01,wv,10
1358803,13,20170726T160859_20170726T161854_T17SMB,-81.954202,37.766073,1501085934130,0.055003,0.027637,0.028270,0.034314,0.049508,...,0.058603,7.0,BELVA COAL COMPANY,00000000000000000869,S009482,2017,2017-07-31,2017-07-01,wv,10
1358804,1,20170726T160859_20170726T161854_T17SMC,-81.102599,38.278113,1501085934130,0.024015,0.767977,0.030674,0.022576,0.035489,...,0.033115,1.0,"CHICOPEE COAL COMPANY, INC.",00000000000000000958,S303686,2017,2017-07-31,2017-07-01,wv,3
1358805,1,20170726T160859_20170726T161854_T17SMC,-81.097209,38.272723,1501085934130,0.024376,0.762291,0.030845,0.022416,0.033265,...,0.030985,1.0,"CHICOPEE COAL COMPANY, INC.",00000000000000000958,S303686,2017,2017-07-31,2017-07-01,wv,3


In [135]:
df = df[['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id', 'State']].drop_duplicates()
df = df.reset_index()
df = df.drop(['index'], axis=1)

### joining aggregated values back into main dataframe

In [136]:
df = pd.merge(df, label0, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label1, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label2, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label3, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label4, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label5, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label6, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label7, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label8, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()

df.fillna(0, inplace=True)
df

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,ky,0.031113,0.623011,0.059563,0.032779,0.065345,0.062902,0.035015,0.032943,0.036427
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,ky,0.031869,0.698788,0.039705,0.029274,0.039489,0.036189,0.033532,0.034271,0.043253
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,ky,0.032205,0.539393,0.055222,0.030944,0.088415,0.041989,0.070858,0.030897,0.037347
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,ky,0.033029,0.735718,0.038324,0.029872,0.033263,0.031299,0.029638,0.032630,0.035219
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,ky,0.030702,0.743186,0.033933,0.027343,0.029957,0.035815,0.029723,0.036228,0.032969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32655,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,wv,0.031039,0.587889,0.039089,0.025979,0.048085,0.042070,0.107876,0.037235,0.042496
32656,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,wv,0.030835,0.739765,0.032717,0.027254,0.030536,0.037661,0.031239,0.037154,0.032554
32657,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,wv,0.027965,0.710045,0.042984,0.025502,0.038912,0.048706,0.032311,0.033129,0.031919
32658,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,wv,0.036899,0.544176,0.040389,0.030385,0.046551,0.055853,0.064093,0.120093,0.039626


### Checking length before and after joining on permit_id

In [141]:
len(df)

32660

In [142]:
pd.merge(df, mines,  how='left', left_on=['orig_perm_id', 'State'], right_on = ['permit_id', 'State'])

,Company,start_date,Observations,orig_perm_id,mine_id,State_x,water,trees,grass,flooded_vegetation,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,State_y
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,ky,0.031113,0.623011,0.059563,0.032779,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,ky,0.031869,0.698788,0.039705,0.029274,...,1984-06-12,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3.0,ky
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,ky,0.032205,0.539393,0.055222,0.030944,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3.0,ky
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,ky,0.033029,0.735718,0.038324,0.029872,...,1975-06-23,2000-07-26,0.0,1.0,0.0,0.0,0.0,0.0,3.0,ky
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,ky,0.030702,0.743186,0.033933,0.027343,...,2001-03-05,2004-05-19,1.0,0.0,0.0,0.0,0.0,0.0,3.0,ky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32665,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,wv,0.031039,0.587889,0.039089,0.025979,...,1998-05-11,NaT,0.0,0.0,0.0,0.0,NaN,0.0,2.0,wv
32666,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,wv,0.030835,0.739765,0.032717,0.027254,...,1989-04-10,NaT,0.0,1.0,0.0,0.0,NaN,1.0,2.0,wv
32667,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,wv,0.027965,0.710045,0.042984,0.025502,...,1985-08-05,NaT,1.0,1.0,0.0,0.0,NaN,1.0,2.0,wv
32668,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,wv,0.036899,0.544176,0.040389,0.030385,...,1982-10-07,NaT,1.0,1.0,1.0,0.0,NaN,1.0,2.0,wv


## 22 duplicates using permit_id

### Appending the number of unique companies per permit_id to dataframe

In [144]:
Company_check = mines[['permit_id', 'company']].groupby(['permit_id']).nunique().sort_values('company', ascending=False)
Company_check = Company_check.rename(columns={"company": "Number_company_to_permit_id"})
Company_check[Company_check['Number_company_to_permit_id']>1]
mines = pd.merge(mines, Company_check,  how='left', left_on=['permit_id'], right_on = ['permit_id'])

### Adding in State column to mines df

In [145]:
mines['State'] = mines['national_id'].str.slice(0, 2)
mines['State'].unique()
mines = mines[mines['State']. isin(['KY', 'TN', 'WV'])]
mines['State'] = mines['State'].str.lower()

<ipython-input-145-5d97319adf63>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mines['State'] = mines['State'].str.lower()


### Checking mines with different company names for the same permit_id

In [146]:
rogue_dupe_mine = mines[(mines['permit_id']=='80-156') & (mines['post_smcra']==1.0)]

In [147]:
rogue_dupe_mine

,company,coalmine_op_status,mine_name,permit_id,state_tribe_mine_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,reported_area,...,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id,State
302,RICHLAND COAL CO,7,AREA 5,80-156,NaN,TN80-156,NaN,NaN,1.0,NaN,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,2.0,tn


In [148]:
mines = mines[(mines['permit_id']!='80-156')]

In [149]:
mines = mines.append(rogue_dupe_mine)

<ipython-input-149-f3ed2ca40002>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mines = mines.append(rogue_dupe_mine)


### Dropping values with missing Company

In [150]:
mines_clean = mines[mines['company'].isnull()]

In [151]:
data_check = pd.merge(df, mines_clean,  how='left', left_on=['orig_perm_id', 'State'], right_on = ['permit_id', 'State'])
data_check = data_check.drop_duplicates()
data_check

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,ky,0.031113,0.623011,0.059563,0.032779,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,ky,0.031869,0.698788,0.039705,0.029274,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,ky,0.032205,0.539393,0.055222,0.030944,...,1899-12-30,1999-11-30,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,ky,0.033029,0.735718,0.038324,0.029872,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,ky,0.030702,0.743186,0.033933,0.027343,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32663,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,wv,0.031039,0.587889,0.039089,0.025979,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32664,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,wv,0.030835,0.739765,0.032717,0.027254,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32665,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,wv,0.027965,0.710045,0.042984,0.025502,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32666,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,wv,0.036899,0.544176,0.040389,0.030385,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
df = pd.merge(df, mines_clean[mines_clean['mine_name'].isnull()],  how='left', left_on=['orig_perm_id', 'State'], right_on = ['permit_id', 'State'])

In [126]:
mines_clean[mines_clean['permit_approval_date'].isnull()]

,company,coalmine_op_status,mine_name,permit_id,state_tribe_mine_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,reported_area,...,permit_approval_date,edit_date,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,State
3310,NaN,4,NaN,458875X,NaN,KY458875X,NaN,NaN,NaN,NaN,...,NaT,2019-01-07,NaN,NaN,NaN,NaN,NaN,NaN,3,ky
3315,NaN,4,NaN,459475X,NaN,KY459475X,NaN,NaN,NaN,NaN,...,NaT,2019-01-07,NaN,NaN,NaN,NaN,NaN,NaN,3,ky
3321,NaN,4,NaN,254671X,NaN,KY254671X,NaN,NaN,NaN,NaN,...,NaT,2019-01-07,NaN,NaN,NaN,NaN,NaN,NaN,3,ky
3322,NaN,4,NaN,459575X,NaN,KY459575X,NaN,NaN,NaN,NaN,...,NaT,2019-01-07,NaN,NaN,NaN,NaN,NaN,NaN,3,ky
3342,NaN,4,NaN,411275X,NaN,KY411275X,NaN,NaN,NaN,NaN,...,NaT,2019-01-07,NaN,NaN,NaN,NaN,NaN,NaN,3,ky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20741,NaN,2,NaN,9180502,NaN,KY9180502,NaN,NaN,0.0,NaN,...,NaT,1999-11-30,0.0,0.0,0.0,0.0,NaN,0.0,3,ky
20747,NaN,2,NaN,8070470,NaN,KY8070470,NaN,NaN,0.0,NaN,...,NaT,1999-11-30,0.0,0.0,0.0,0.0,NaN,0.0,3,ky
20771,NaN,2,NaN,8070370,NaN,KY8070370,NaN,NaN,0.0,NaN,...,NaT,1999-11-30,0.0,0.0,0.0,0.0,NaN,0.0,3,ky
21191,NaN,0,NaN,NaN,NaN,WV,NaN,NaN,1.0,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2,wv


### Creating empty rows for missing years

In [155]:
all_year_mines = df['mine_id'][df['start_date']=='2017-07-01'].unique()
appenddf = df[['Company', 'mine_id']].query('mine_id not in @all_year_mines')
appenddf = appenddf.drop_duplicates()
appenddf['start_date']='2017-07-01'
appenddf['start_date'] = pd.to_datetime(appenddf['start_date'])
appenddf['end_date']='2017-07-31'
appenddf['end_date'] = pd.to_datetime(appenddf['end_date'])
df = pd.concat([df, appenddf], ignore_index=True)

all_year_mines = df['mine_id'][df['start_date']=='2019-07-01'].unique()
appenddf = df[['Company', 'mine_id']].query('mine_id not in @all_year_mines')
appenddf = appenddf.drop_duplicates()
appenddf['start_date']='2019-07-01'
appenddf['start_date'] = pd.to_datetime(appenddf['start_date'])
appenddf['end_date']='2019-07-31'
appenddf['end_date'] = pd.to_datetime(appenddf['end_date'])
df = pd.concat([df, appenddf], ignore_index=True)

all_year_mines = df['mine_id'][df['start_date']=='2021-07-01'].unique()
appenddf = df[['Company', 'mine_id']].query('mine_id not in @all_year_mines')
appenddf = appenddf.drop_duplicates()
appenddf['start_date']='2021-07-01'
appenddf['start_date'] = pd.to_datetime(appenddf['start_date'])
appenddf['end_date']='2021-07-31'
appenddf['end_date'] = pd.to_datetime(appenddf['end_date'])
df = pd.concat([df, appenddf], ignore_index=True)

### Removing any rows missing 2023

In [156]:
all_year_mines = df['mine_id'][df['start_date']=='07-01-2023'].unique()
df = df.query('mine_id in @all_year_mines')

In [157]:
file_name = 'all_mines_no_filter.csv'
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>